In [1]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

/home/development/raghava/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
df2 = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/test.tsv', delimiter='\t', header=None)

In [6]:
df2

,0,1
0,"no movement , no yuks , not much of anything",0
1,"a gob of drivel so sickly sweet , even the eag...",0
2,"gangs of new york is an unapologetic mess , wh...",0
3,"we never really feel involved with the story ,...",0
4,this is one of polanski 's best films,1
...,...,...
1816,"an often deadly boring , strange reading of a ...",0
1817,the problem with concept films is that if the ...,0
1818,"safe conduct , however ambitious and well inte...",0
1819,"a film made with as little wit , interest , an...",0


In [7]:
df.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [10]:
from transformers import AutoTokenizer, DistilBertModel
import torch

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
input = df.iloc[0][0]
ids = tokenizer(input,add_special_tokens = True,return_tensors  = 'pt',padding=True,truncation=True)
print(ids)
output = model(**ids)
last_layer = output.last_hidden_state
last_layer

{'input_ids': tensor([[  101,  1037, 18385,  1010,  6057,  1998,  2633, 18276,  2128, 16603,
          1997,  5053,  1998,  1996,  6841,  1998,  5687,  5469,  3152,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


tensor([[[-0.2159, -0.1403,  0.0083,  ..., -0.1369,  0.5867,  0.2011],
         [-0.2471,  0.2468,  0.1008,  ..., -0.1631,  0.9349, -0.0715],
         [ 0.0558,  0.3573,  0.4140,  ..., -0.2430,  0.1770, -0.5080],
         ...,
         [ 0.5113,  0.3471,  0.0273,  ...,  0.3259,  0.7047, -0.3381],
         [ 0.2807,  0.5242, -0.5708,  ...,  0.4081,  0.4943, -0.0218],
         [ 0.8637,  0.3259, -0.1457,  ...,  0.0325, -0.4152, -0.1499]]],
       grad_fn=<NativeLayerNormBackward0>)

In [12]:
tokenized = df[0].apply((lambda x: tokenizer(x,add_special_tokens = True,return_tensors  = 'pt',padding = True)))

In [13]:
tokenized[0]

{'input_ids': tensor([[  101,  1037, 18385,  1010,  6057,  1998,  2633, 18276,  2128, 16603,
          1997,  5053,  1998,  1996,  6841,  1998,  5687,  5469,  3152,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [14]:
# lis = []
# for ele in tokenized:
#     lis.append(len(ele['input_ids'][0]))
# max(lis)
max_leng = max([len(ele['input_ids'][0]) for ele in tokenized])
max_leng

67

In [15]:
for ele in tokenized.values:
    pad = torch.zeros(max_leng-len(ele['input_ids'][0])).int()
    temp = torch.cat((ele['input_ids'][0],pad),0)
    ele['input_ids'] = temp[None,:]
    temp = torch.cat((ele['attention_mask'][0],pad),0)
    ele['attention_mask'] = temp[None,:]

In [16]:
with torch.no_grad():
    last = model(**tokenized[0])
    last_h = last.last_hidden_state
    cls = last_h[0][0]
    for ind,ele in enumerate(tokenized):
        if ind==0:
            continue
        last = model(**ele)
        last_h = last.last_hidden_state
        cls = torch.vstack((cls,last_h[0][0]))
        if ind%100 == 0:
            print(ind)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900


In [17]:
cls.shape, cls

(torch.Size([6920, 768]),
 tensor([[-0.2159, -0.1403,  0.0083,  ..., -0.1369,  0.5867,  0.2011],
         [-0.1726, -0.1448,  0.0022,  ..., -0.1744,  0.2139,  0.3720],
         [-0.0506,  0.0720, -0.0296,  ..., -0.0715,  0.7185,  0.2623],
         ...,
         [-0.0655, -0.0518, -0.1409,  ..., -0.0645,  0.6022,  0.2135],
         [-0.0852, -0.0487, -0.0814,  ..., -0.1359,  0.3951,  0.2289],
         [-0.2944, -0.0923, -0.0083,  ..., -0.0516,  0.4350,  0.2889]]))

In [26]:
# add this random dimensionality reduction
# cls1 = cls[:,0:64]

In [27]:
features = cls.numpy()
features,features.shape

(array([[-0.21593432, -0.14028907,  0.00831073, ..., -0.13694835,
          0.58670044,  0.20112701],
        [-0.17262723, -0.14476171,  0.00223433, ..., -0.17442548,
          0.21386431,  0.3719748 ],
        [-0.05063358,  0.07203972, -0.02959727, ..., -0.07148945,
          0.718524  ,  0.26225477],
        ...,
        [-0.06550945, -0.05184749, -0.14094478, ..., -0.0645067 ,
          0.6022304 ,  0.21347874],
        [-0.08523133, -0.04869818, -0.08137523, ..., -0.1358935 ,
          0.39505628,  0.22889718],
        [-0.2943684 , -0.09234691, -0.00831687, ..., -0.0515911 ,
          0.43497857,  0.2889159 ]], dtype=float32),
 (6920, 768))

In [28]:
labels = df[1]
labels

0       1
1       0
2       0
3       1
4       1
       ..
6915    1
6916    0
6917    0
6918    0
6919    1
Name: 1, Length: 6920, dtype: int64

In [29]:
train_ft,test_ft, train_lb, test_lb = train_test_split(features,labels)

In [30]:
lr_clf = LogisticRegression()
lr_clf.fit(train_ft, train_lb)

/home/development/raghava/env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [31]:
lr_clf.score(test_ft,test_lb)

0.8502890173410405